# Comparision of songs by two singers
In this notebook, the songs by two different singers are compared and graphed to determine the similarity between them. This is achieved using the MusiCNN model to extract embeddings from the mel spectograms of the mp3 files, and comparing the similarities between the embeddings of the songs. 

### Step 1: Import necessary libraries

In [1]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import json

2024-06-21 01:19:54.827153: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-21 01:19:54.866675: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 01:19:55.456905: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Step 2: Define paths to model and dataset

In [2]:
MUSICNN_SR = 16000
DATASET_DIR = '../dataset/singers_mp3'
MODEL_NAME = '../model/genre_tzanetakis-musicnn-msd-1'
MODEL_JSON = f'{MODEL_NAME}.json'
MODEL_PB = f'{MODEL_NAME}.pb'

musicnn_metadata = json.load(open(MODEL_JSON, 'r'))
for k, v in musicnn_metadata.items():
    print('{}: {}'.format(k , v))

name: genre GTZAN
type: multi-class classifier
link: https://essentia.upf.edu/models/classifiers/genre_tzanetakis/genre_tzanetakis-musicnn-msd-1.pb
version: 1
description: classification of music by genre
author: Pablo Alonso
email: pablo.alonso@upf.edu
release_date: 2020-03-31
framework: tensorflow
framework_version: 1.15.0
classes: ['blu', 'cla', 'cou', 'dis', 'hip', 'jaz', 'met', 'pop', 'reg', 'roc']
model_types: ['frozen_model']
dataset: {'name': 'the GTZAN Genre Collection', 'citation': '@article{tzanetakis2002musical,\n  title={Musical genre classification of audio signals},\n  author={Tzanetakis, George and Cook, Perry},\n  journal={IEEE Transactions on speech and audio processing},\n  volume={10},\n  number={5},\n  pages={293--302},\n  year={2002},\n  publisher={IEEE}\n}', 'size': '1000 track excerpts, 100 per genre', 'metrics': {'5-fold_cross_validation_normalized_accuracy': 0.83}}
schema: {'inputs': [{'name': 'model/Placeholder', 'type': 'float', 'shape': [187, 96]}], 'output

We can observe the output of the penultimate dense layer is proposed as embeddings. 
We will use it to extract songs embeddings from our dataset.

### Step 3: Functions for loading and processing audio

In [3]:
def load_model(pb_file_path):
    with tf.io.gfile.GFile(pb_file_path, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.compat.v1.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="")
    return graph

def preprocess_audio(file_path, sample_rate=MUSICNN_SR):
    y, sr = librosa.load(file_path, sr=sample_rate)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=96, n_fft=2048, hop_length=512)
    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    if log_mel_spec.shape[1] > 187:
        log_mel_spec = log_mel_spec[:, :187]
    else:
        pad_width = 187 - log_mel_spec.shape[1]
        log_mel_spec = np.pad(log_mel_spec, ((0, 0), (0, pad_width)), mode='constant')
    return log_mel_spec.T[np.newaxis, :, :]

def run_model(graph, input_data):
    input_tensor = graph.get_tensor_by_name('model/Placeholder:0')
    output_tensor = graph.get_tensor_by_name('model/dense/BiasAdd:0')
    with tf.compat.v1.Session(graph=graph) as sess:
        embeddings = sess.run(output_tensor, feed_dict={input_tensor: input_data})
    return embeddings

def extract_mean_embedding(filename):
    audio_data = preprocess_audio(filename)
    graph = load_model(MODEL_PB)
    embeddings = run_model(graph, audio_data)
    return embeddings

def process_dataset(dataset_dir):
    embeddings = []
    for root, dirs, files in os.walk(dataset_dir):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            print(f"Directory processing : {dir_path}")
            for root2, dirs2, files2 in os.walk(dir_path):
                for file in files2:
                    file_path = os.path.join(root2, file)
                    mean_embedding = extract_mean_embedding(file_path)
                    embeddings.append((file_path, mean_embedding))
    return embeddings

### Step 4: Process dataset and extract embeddings

In [4]:
file_embeddings = process_dataset(DATASET_DIR)
embeddings = []
for file_path, embedding in file_embeddings:
    print(f'File path: {file_path}')
    embeddings.append(embedding)

print(f'There are {len(embeddings)} songs in this dataset.')

embeddings = np.array(embeddings)
embeddings_matrix = np.vstack([emb for emb in embeddings]) 
similarities = cosine_similarity(embeddings_matrix)

Directory processing : ../dataset/singers_mp3/SPB


2024-06-21 01:20:01.586966: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-21 01:20:01.635925: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-21 01:20:01.641056: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 3215270.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/

Directory processing : ../dataset/singers_mp3/vocals2
Directory processing : ../dataset/singers_mp3/only_vocals_1
Directory processing : ../dataset/singers_mp3/vocals
Directory processing : ../dataset/singers_mp3/sid_sriram


Note: Illegal Audio-MPEG-Header 0x00000000 at offset 6412288.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
/tmp/ipykernel_1780361/846677368.py:10: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=sample_rate)
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipykernel_1780361/846677368.py:10: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=sample_rate)
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa

Directory processing : ../dataset/singers_mp3/only_vocals_2


KeyboardInterrupt: 

### Step 5: Plot heatmap and standardize similarities

In [ ]:
print(similarities.shape)
similarities = similarities.round(5)
print(f'Min value: {similarities.min()}')
print(f'Max value: {similarities.max()}')
df_similarities = pd.DataFrame(similarities)
fig = px.imshow(similarities, color_continuous_scale='RdYlGn')
fig.show()

(80, 80)
Min value: 0.7085199952125549
Max value: 1.0


After standardizing the values between 0.85 and 1, and normalizing them: 

In [ ]:
y = np.array(similarities)

for i in range(y.shape[0]):
    for j in range(y.shape[1]):
        if(y[i][j]<0.85):
            y[i][j] = 0.85

print(f'Min value: {y.min()}')
print(f'Max value: {y.max()}')
df_similarities = pd.DataFrame(y)
# df_similarities.to_csv('../data/similarity_data_singers_std.csv', index=False)
fig = px.imshow(y, color_continuous_scale='RdYlGn')
fig.show()

Min value: 0.8500000238418579
Max value: 1.0


### Step 6: Apply demucs to separate vocals

In [2]:
import os
import subprocess

input_dir = '../dataset/singers_mp3/sid_sriram'
output_dir = '../dataset/singers_mp3/vocals'

mp3_files = [f for f in os.listdir(input_dir)]

for mp3_file in mp3_files:
    mp3_path = os.path.join(input_dir, mp3_file)
    song_name = os.path.splitext(mp3_file)[0]
    song_output_dir = os.path.join(output_dir, song_name)
    if not os.path.exists(song_output_dir):
        os.makedirs(song_output_dir)
    command = ['demucs', mp3_path, '-o', song_output_dir]
    subprocess.run(command)

print("Separation complete.")

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/rohitsangubotla/Documents/Projects/Music_Similarity/dataset/singers_mp3/vocals/Naa Kanulu Yepudu __ SenSongsMp3.Com/htdemucs
Separating track ../dataset/singers_mp3/sid_sriram/Naa Kanulu Yepudu __ SenSongsMp3.Com.mp3


  0%|                                                                                  | 0.0/257.4 [00:00<?, ?seconds/s]/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.v

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/rohitsangubotla/Documents/Projects/Music_Similarity/dataset/singers_mp3/vocals/Ee Maya Peremita - SenSongsMp3.Co/htdemucs
Separating track ../dataset/singers_mp3/sid_sriram/Ee Maya Peremita - SenSongsMp3.Co.mp3


  0%|                                                                     | 0.0/251.54999999999998 [00:00<?, ?seconds/s]/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.v

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/rohitsangubotla/Documents/Projects/Music_Similarity/dataset/singers_mp3/vocals/02 - Hey Idi Nenena - SenSongsmp3.Co/htdemucs
Separating track ../dataset/singers_mp3/sid_sriram/02 - Hey Idi Nenena - SenSongsmp3.Co.mp3


...

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/rohitsangubotla/Documents/Projects/Music_Similarity/dataset/singers_mp3/vocals/Choosale Kallaraa - SenSongsMp3.Com/htdemucs
Separating track ../dataset/singers_mp3/sid_sriram/Choosale Kallaraa - SenSongsMp3.Com.mp3


  0%|                                                                     | 0.0/228.14999999999998 [00:00<?, ?seconds/s]/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.v

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/rohitsangubotla/Documents/Projects/Music_Similarity/dataset/singers_mp3/vocals/Bagundhi Ee Kaalame/htdemucs
Separating track ../dataset/singers_mp3/sid_sriram/Bagundhi Ee Kaalame.mp3


  0%|                                                                                  | 0.0/234.0 [00:00<?, ?seconds/s]/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.v

Separation complete.


In [3]:
import os
import subprocess

input_dir = '../dataset/singers_mp3/SPB'
output_dir = '../dataset/singers_mp3/vocals2'

mp3_files = [f for f in os.listdir(input_dir)]

for mp3_file in mp3_files:
    mp3_path = os.path.join(input_dir, mp3_file)
    song_name = os.path.splitext(mp3_file)[0]
    song_output_dir = os.path.join(output_dir, song_name)
    if not os.path.exists(song_output_dir):
        os.makedirs(song_output_dir)
    command = ['demucs', mp3_path, '-o', song_output_dir]
    subprocess.run(command)

print("Separation complete.")

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/rohitsangubotla/Documents/Projects/Music_Similarity/dataset/singers_mp3/vocals2/[iSongs.info] 03 - Rangulalo/htdemucs
Separating track ../dataset/singers_mp3/SPB/[iSongs.info] 03 - Rangulalo.mp3


  0%|                                                                                 | 0.0/274.95 [00:00<?, ?seconds/s]/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.v

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/rohitsangubotla/Documents/Projects/Music_Similarity/dataset/singers_mp3/vocals2/[iSongs.info] 02 - Manishiko Sneham/htdemucs
Separating track ../dataset/singers_mp3/SPB/[iSongs.info] 02 - Manishiko Sneham.mp3


  0%|                                                                                 | 0.0/263.25 [00:00<?, ?seconds/s]/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.v

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/rohitsangubotla/Documents/Projects/Music_Similarity/dataset/singers_mp3/vocals2/[iSongs.info] 12 - Sagiraa Thoduga/htdemucs
Separating track ../dataset/singers_mp3/SPB/[iSongs.info] 12 - Sagiraa Thoduga.mp3


...

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/rohitsangubotla/Documents/Projects/Music_Similarity/dataset/singers_mp3/vocals2/[iSongs.info] 06 - Idhi Ganavijayam/htdemucs
Separating track ../dataset/singers_mp3/SPB/[iSongs.info] 06 - Idhi Ganavijayam.mp3


  0%|                                                                                  | 0.0/257.4 [00:00<?, ?seconds/s]/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.v

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/rohitsangubotla/Documents/Projects/Music_Similarity/dataset/singers_mp3/vocals2/[iSongs.info] 07 - Odhante Vinade/htdemucs
Separating track ../dataset/singers_mp3/SPB/[iSongs.info] 07 - Odhante Vinade.mp3


  0%|                                                                                  | 0.0/304.2 [00:00<?, ?seconds/s]/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/home/rohitsangubotla/Documents/Projects/Music_Similarity/.v

Separation complete.


In [4]:
import os
import shutil

SEPARATED_DIR_1 = '../dataset/singers_mp3/vocals'
SEPERATED_DIR_2 = '../dataset/singers_mp3/vocals2'

TARGET_DIRS_1 = {
    'bass': '../dataset/rest',
    'drums': '../dataset/rest',
    'other': '../dataset/rest',
    'vocals': '../dataset/singers_mp3/only_vocals_1'
}
TARGET_DIRS_2 = {
    'bass': '../dataset/rest',
    'drums': '../dataset/rest',
    'other': '../dataset/rest',
    'vocals': '../dataset/singers_mp3/only_vocals_2'
}

def seperate(seperated_dir,target_dirs):
    for target_dir in target_dirs.values():
        os.makedirs(target_dir, exist_ok=True)
    for song_dir in os.listdir(seperated_dir):
        song_path = os.path.join(seperated_dir, song_dir)
        if os.path.isdir(song_path):
            for file_type in target_dirs.keys():
                source_file = os.path.join(song_path,'htdemucs',song_dir, f'{file_type}.wav')
                print(source_file)
                if os.path.exists(source_file):
                    target_file = os.path.join(target_dirs[file_type], f'{song_dir}_{file_type}.wav')
                    print(target_file)
                    shutil.move(source_file, target_file)

seperate(SEPARATED_DIR_1,TARGET_DIRS_1)
seperate(SEPERATED_DIR_2,TARGET_DIRS_2)

../dataset/singers_mp3/vocals/[iSongs.info] 01 - Maguva Maguva/htdemucs/[iSongs.info] 01 - Maguva Maguva/bass.wav
../dataset/rest/[iSongs.info] 01 - Maguva Maguva_bass.wav
../dataset/singers_mp3/vocals/[iSongs.info] 01 - Maguva Maguva/htdemucs/[iSongs.info] 01 - Maguva Maguva/drums.wav
../dataset/rest/[iSongs.info] 01 - Maguva Maguva_drums.wav
../dataset/singers_mp3/vocals/[iSongs.info] 01 - Maguva Maguva/htdemucs/[iSongs.info] 01 - Maguva Maguva/other.wav
../dataset/rest/[iSongs.info] 01 - Maguva Maguva_other.wav
../dataset/singers_mp3/vocals/[iSongs.info] 01 - Maguva Maguva/htdemucs/[iSongs.info] 01 - Maguva Maguva/vocals.wav
../dataset/singers_mp3/only_vocals_1/[iSongs.info] 01 - Maguva Maguva_vocals.wav
../dataset/singers_mp3/vocals/Maate Vinadhuga - SenSongsMp3.Co/htdemucs/Maate Vinadhuga - SenSongsMp3.Co/bass.wav
../dataset/rest/Maate Vinadhuga - SenSongsMp3.Co_bass.wav
../dataset/singers_mp3/vocals/Maate Vinadhuga - SenSongsMp3.Co/htdemucs/Maate Vinadhuga - SenSongsMp3.Co/drums.

### Step 7: Run steps 3-5 on the vocal only data

In [5]:
def process_dataset(dataset_dir):
    embeddings = []
    for root, dirs, files in os.walk(dataset_dir):
        print(len(files))
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                mean_embedding = extract_mean_embedding(file_path)
                embeddings.append((file_path, mean_embedding))
    return embeddings

embeddings = []
file_embeddings = process_dataset('../dataset/singers_mp3/only_vocals_1')
for file_path, embedding in file_embeddings:
    print(f'File path: {file_path}')
    embeddings.append(embedding)

file_embeddings = process_dataset('../dataset/singers_mp3/only_vocals_2')
for file_path, embedding in file_embeddings:
    print(f'File path: {file_path}')
    embeddings.append(embedding)

print(f'There are {len(embeddings)} songs in this dataset.')

embeddings = np.array(embeddings)
embeddings_matrix = np.vstack([emb for emb in embeddings]) 
similarities = cosine_similarity(embeddings_matrix)

40
File path: ../dataset/singers_mp3/only_vocals_1/02 - Hey Idi Nenena - SenSongsmp3.Co_vocals.wav
File path: ../dataset/singers_mp3/only_vocals_1/Undiporaadhey - SenSongsMp3.Co_vocals.wav
File path: ../dataset/singers_mp3/only_vocals_1/Samajavaragamana - SenSongsMp3.Co_vocals.wav
File path: ../dataset/singers_mp3/only_vocals_1/Ninne Ninne - SenSongsMp3.Co_vocals.wav
File path: ../dataset/singers_mp3/only_vocals_1/Naa Chelive_vocals.wav
File path: ../dataset/singers_mp3/only_vocals_1/Urike-Urike(PaglaSongs)_vocals.wav
File path: ../dataset/singers_mp3/only_vocals_1/Baguntundi-Nuvvu-Navvithe(PaglaSongs)_vocals.wav
File path: ../dataset/singers_mp3/only_vocals_1/Maate Vinadhuga - SenSongsMp3.Co_vocals.wav
File path: ../dataset/singers_mp3/only_vocals_1/Vulikithe Vudhyamam - SenSongsMp3.Co_vocals.wav
File path: ../dataset/singers_mp3/only_vocals_1/Aagave Nuvvagave_vocals.wav
File path: ../dataset/singers_mp3/only_vocals_1/Bagundhi Ee Kaalame_vocals.wav
File path: ../dataset/singers_mp3/on

In [6]:
print(similarities.shape)
similarities = similarities.round(5)
print(f'Min value: {similarities.min()}')
print(f'Max value: {similarities.max()}')
df_similarities = pd.DataFrame(similarities)
fig = px.imshow(similarities, color_continuous_scale='RdYlGn')
fig.show()

(80, 80)
Min value: 0.870639979839325
Max value: 1.0


In [7]:
df_similarities.to_csv('../data/similarity_data_vocals.csv', index=False)